In [1]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ruta_csv = '/content/drive/MyDrive/CDD/7mo/Bases/MUBI/CSV_FILES/Relacional/'
ruta_columnar = '/content/drive/MyDrive/CDD/7mo/Bases/MUBI/CSV_FILES/Columnar/'

In [56]:
vistas = pd.read_csv(ruta_csv+'Vistas.csv')
resenias = pd.read_csv(ruta_csv+'Resenias.csv')
peliculas = pd.read_csv(ruta_csv+'Peliculas.csv')

# Tabla de Hechos

## Vistas

In [64]:
vistas_hechos = vistas.merge(peliculas[['id_pelicula', 'id_productora', 'id_pais']], how='left', on='id_pelicula')

In [65]:
vistas_hechos.sample()

,id_vista,id_usuario,id_pelicula,fecha,id_productora,id_pais
182329,4758791,49148.0,148320.0,2008-07-12,10193,3


In [66]:
vistas_hechos['fecha'] = pd.to_datetime( vistas_hechos['fecha'] )

In [68]:
vistas_hechos['anio'] = vistas_hechos['fecha'].apply(lambda x: x.year)
vistas_hechos['mes'] = vistas_hechos['fecha'].apply(lambda x: x.month)

In [69]:
vistas_hechos = vistas_hechos.drop(['id_vista', 'id_usuario'], axis = 1)

In [70]:
vistas_hechos = vistas_hechos.groupby(by=['id_productora', 'id_pais', 'id_pelicula', 'anio', 'mes']).count()

In [71]:
vistas_hechos.describe()

,fecha
count,1.395097e+06
mean,3.583980e+00
std,2.438428e+00
min,1.000000e+00
25%,1.000000e+00
50%,3.000000e+00
75%,5.000000e+00
max,2.000000e+01


In [79]:
vistas_hechos = vistas_hechos.rename({'fecha': 'vistas'}, axis = 1)
vistas_hechos

vistas
id_productora id_pais id_pelicula anio mes        
9             6       137198.0    2011 2         1
                                       8         1
                                  2015 7         1
                                  2018 10        1
                                  2019 6         1
...                                            ...
13586         52      159077.0    2009 5         1
13700         8       159334.0    2016 12        1
13705         15      159342.0    2013 8         1
13785         9       159528.0    2019 8         1
13939         2       159906.0    2017 10        1

[1395097 rows x 1 columns]

In [81]:
vistas_hechos = vistas_hechos.reset_index()

In [82]:
vistas_hechos.to_csv(ruta_columnar + 'Vistas.csv', index=False)

# Reseñas

In [102]:
resenias_hechos = resenias.merge(peliculas[['id_pelicula', 'id_productora', 'id_pais']], how='left', on='id_pelicula')

In [103]:
resenias_hechos.sample()

,id_resenia,id_usuario,id_pelicula,calificacion,comentario,fecha,id_productora,id_pais
426855,14041360,24530959,104590,5.0,Sesak nafas!,2016-05-21,42.0,2.0


In [104]:
resenias_hechos['fecha'] = pd.to_datetime( resenias_hechos['fecha'] )

In [105]:
resenias_hechos['anio'] = resenias_hechos['fecha'].apply(lambda x: x.year)
resenias_hechos['mes'] = resenias_hechos['fecha'].apply(lambda x: x.month)

In [106]:
resenias_hechos = resenias_hechos.drop(['id_resenia', 'id_usuario', 'fecha'], axis = 1)

In [107]:
resenias_hechos = resenias_hechos.groupby(by=['id_productora', 'id_pais', 'id_pelicula', 'anio', 'mes']).agg({'comentario':'count', 'calificacion':'mean'})

In [108]:
resenias_hechos.describe()

,comentario,calificacion
count,416031.000000,400595.000000
mean,1.799126,3.380574
std,3.320511,1.101769
min,1.000000,1.000000
25%,1.000000,3.000000
50%,1.000000,3.444444
75%,2.000000,4.000000
max,276.000000,5.000000


In [109]:
resenias_hechos

comentario  calificacion
id_productora id_pais id_pelicula anio mes                          
0.0           0.0     1           2008 5             1           NaN
                                       11            1           4.0
                                       12            1           4.0
                                  2009 12            1           5.0
                                  2010 1             1           5.0
...                                                ...           ...
30118.0       19.0    110366      2018 2             1           4.0
                                       6             1           3.0
                                       9             1           5.0
                                  2019 1             1           3.0
30120.0       19.0    198348      2018 6             1           2.0

[416031 rows x 2 columns]

In [110]:
resenias_hechos.columns = ['comentarios', 'calificacion']

In [111]:
resenias_hechos = resenias_hechos.reset_index()

In [114]:
resenias_hechos.describe()

,id_productora,id_pais,id_pelicula,anio,mes,comentarios,calificacion
count,416031.000000,416031.000000,416031.000000,416031.000000,416031.000000,416031.000000,400595.000000
mean,4684.786941,9.239564,55583.004519,2015.636313,6.185501,1.799126,3.380574
std,7345.955316,14.040457,61042.139558,2.991091,3.578291,3.320511,1.101769
min,0.000000,0.000000,1.000000,2008.000000,1.000000,1.000000,1.000000
25%,146.000000,2.000000,3778.000000,2014.000000,3.000000,1.000000,3.000000
50%,780.000000,2.000000,28156.000000,2016.000000,6.000000,1.000000,3.444444
75%,5977.000000,15.000000,102145.500000,2018.000000,9.000000,2.000000,4.000000
max,30120.000000,157.000000,263689.000000,2020.000000,12.000000,276.000000,5.000000


In [116]:
resenias_hechos.to_csv(ruta_columnar + 'Resenias.csv', index = False)